In [1]:
import os 
import pandas as pd 
import numpy as np
dataDir = r'C:\Users\jihhwan\sum_all_processing\total_raw(n=58)'
file_nms = os.listdir(dataDir)
file_nms_csv = [file for file in file_nms if file.endswith(".csv")] 
sumDF_rt = pd.DataFrame(columns=['SMI','SMO','SMN','SEI','SEO','SEN','MMI','MMO','MMN','MEI','MEO','MEN','LMI','LMO','LMN','LEI','LEO','LEN'])
DF_18condition = pd.DataFrame(columns=['SMI','SMO','SMN','SEI','SEO','SEN','MMI','MMO','MMN','MEI','MEO','MEN','LMI','LMO','LMN','LEI','LEO','LEN'])
DF_resource_inequity = pd.DataFrame(columns=['SM','SE','MM','ME','LM','LE'])
DF_resource_group = pd.DataFrame(columns=['SI','SO','SN','MI','MO','MN','LI','LO','LN'])
DF_inequity_group = pd.DataFrame(columns=['ModI','ModO','ModN','EI','EO','EN'])
DF_maineffect = pd.DataFrame(columns=['Small','Medium','Large','Moderate','Extreme','Ingroup','Outgroup', 'Neutral'])
DF_combination = pd.DataFrame(columns=['ingroup_bias', 'inequity_aversion', 'interaction', 'overall_accept'])


for fileName in file_nms_csv:
    total_dir = os.path.join(dataDir, fileName)
    
    csv_data = pd.read_csv(total_dir)
    case1 = csv_data.iloc[11:251:2,:6]
    case2 = csv_data.iloc[11:251:2,110]
    case3 = csv_data.iloc[11:251:2,111] 
    actual_output = pd.concat([case1, case2, case3], axis=1)
    actual_output.loc[(actual_output['par_left'] == 'B1')|(actual_output['par_left'] =='B2')|(actual_output['par_left'] =='B3'), 'par_Left']='B'
    actual_output.loc[(actual_output['par_left'] == 'A1')|(actual_output['par_left'] =='A2'), 'par_Left']='A'
    actual_output.loc[(actual_output['par_right'] == 'B1')|(actual_output['par_right'] =='B2')|(actual_output['par_right'] =='B3'), 'par_Right']='B'
    actual_output.loc[(actual_output['par_right'] == 'A1')|(actual_output['par_right'] =='A2'), 'par_Right']='A'
    #A, B 변환 

    actual_output.loc[actual_output['resource_level'] == 25.0, 'resource_level_1'] =3
    actual_output.loc[actual_output['resource_level'] == 20.0, 'resource_level_1'] =2
    actual_output.loc[actual_output['resource_level'] == 15.0, 'resource_level_1'] =1
    # 3 = 25, 2 = 20, 1 = 15

    actual_output.loc[actual_output['inequity_level_left'] == 6.0, 'inequity_level']=1
    actual_output.loc[actual_output['inequity_level_left'] == 8.0, 'inequity_level']=1
    actual_output.loc[actual_output['inequity_level_left'] == 10.0, 'inequity_level']=1
    actual_output.loc[actual_output['inequity_level_left'] == 3.0, 'inequity_level']=2
    actual_output.loc[actual_output['inequity_level_left'] == 4.0, 'inequity_level']=2
    actual_output.loc[actual_output['inequity_level_left'] == 5.0, 'inequity_level']=2
    # 1 = 6:4, 2 = 8:2 

    actual_output.loc[(actual_output['par_Right'] == 'A')&(actual_output['par_Left'] =='B'), 'who_adv']=1  #advantageous equity =1 
    actual_output.loc[(actual_output['par_Right'] == 'B')&(actual_output['par_Left'] =='A'), 'who_adv']=2  #disadcantageous equity = 2
    actual_output.loc[(actual_output['par_Right'] == 'A')&(actual_output['par_Left'] =='A'), 'who_adv']=3  #same = 3
    actual_output.loc[(actual_output['par_Right'] == 'B')&(actual_output['par_Left'] =='B'), 'who_adv']=3

    actual_output.loc[(actual_output['key_resp_2.keys'] == 'n'), 'response']=0
    actual_output.loc[(actual_output['key_resp_2.keys'] == 'y'), 'response']=1

    actual_output['reaction_time'] = actual_output['key_resp_2.rt']

    #print(actual_output.head())
    
    refined_data = actual_output.iloc[:,10:]
    csv_data = refined_data    #refining one participant's data 
    
    #trimming out extreme data 
    t = csv_data['reaction_time'].tolist()
    
        #median calculation
    M = np.median(t)
    value  = []
    for i in t:
        a = abs(i - M)
        value.append(a)

    Median = np.median(value)*1.4826   
    csv_data = csv_data[csv_data['reaction_time'] < M+ 3*Median]
    
    #zScoring
    csv_data['zScore'] = 0
    
    for i in range(0,len(csv_data)):
        csv_data['zScore'].iloc[i] =  (csv_data.iloc[i]['reaction_time']-csv_data['reaction_time'].mean())/csv_data['reaction_time'].std()
        
        
        
    
    #labelling condition 
    SMI = csv_data.loc[(csv_data['resource_level_1'] == 1.0)&(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 1.0)].mean()
    SMO = csv_data.loc[(csv_data['resource_level_1'] == 1.0)&(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 2.0)].mean()
    SMN = csv_data.loc[(csv_data['resource_level_1'] == 1.0)&(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 3.0)].mean()

    SEI = csv_data.loc[(csv_data['resource_level_1'] == 1.0)&(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 1.0)].mean()
    SEO = csv_data.loc[(csv_data['resource_level_1'] == 1.0)&(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 2.0)].mean()
    SEN = csv_data.loc[(csv_data['resource_level_1'] == 1.0)&(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 3.0)].mean()

    MMI = csv_data.loc[(csv_data['resource_level_1'] == 2.0)&(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 1.0)].mean()
    MMO = csv_data.loc[(csv_data['resource_level_1'] == 2.0)&(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 2.0)].mean()
    MMN = csv_data.loc[(csv_data['resource_level_1'] == 2.0)&(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 3.0)].mean()

    MEI = csv_data.loc[(csv_data['resource_level_1'] == 2.0)&(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 1.0)].mean()
    MEO = csv_data.loc[(csv_data['resource_level_1'] == 2.0)&(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 2.0)].mean()
    MEN = csv_data.loc[(csv_data['resource_level_1'] == 2.0)&(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 3.0)].mean()

    LMI = csv_data.loc[(csv_data['resource_level_1'] == 3.0)&(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 1.0)].mean()
    LMO = csv_data.loc[(csv_data['resource_level_1'] == 3.0)&(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 2.0)].mean()
    LMN = csv_data.loc[(csv_data['resource_level_1'] == 3.0)&(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 3.0)].mean()

    LEI = csv_data.loc[(csv_data['resource_level_1'] == 3.0)&(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 1.0)].mean()
    LEO = csv_data.loc[(csv_data['resource_level_1'] == 3.0)&(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 2.0)].mean()
    LEN = csv_data.loc[(csv_data['resource_level_1'] == 3.0)&(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 3.0)].mean()
    

    #resource_inequity 3x2
    SM = csv_data.loc[(csv_data['resource_level_1'] == 1.0)&(csv_data['inequity_level'] == 1.0)].mean()
    SE = csv_data.loc[(csv_data['resource_level_1'] == 1.0)&(csv_data['inequity_level'] == 2.0)].mean()
    
    MM = csv_data.loc[(csv_data['resource_level_1'] == 2.0)&(csv_data['inequity_level'] == 1.0)].mean()
    ME = csv_data.loc[(csv_data['resource_level_1'] == 2.0)&(csv_data['inequity_level'] == 2.0)].mean()
    
    LM = csv_data.loc[(csv_data['resource_level_1'] == 3.0)&(csv_data['inequity_level'] == 1.0)].mean()
    LE = csv_data.loc[(csv_data['resource_level_1'] == 3.0)&(csv_data['inequity_level'] == 2.0)].mean()
    
    #resource_group 3x3 
    SI = csv_data.loc[(csv_data['resource_level_1'] == 1.0)&(csv_data['who_adv'] == 1.0)].mean()
    SO = csv_data.loc[(csv_data['resource_level_1'] == 1.0)&(csv_data['who_adv'] == 2.0)].mean()
    SN = csv_data.loc[(csv_data['resource_level_1'] == 1.0)&(csv_data['who_adv'] == 3.0)].mean()
    
    MI = csv_data.loc[(csv_data['resource_level_1'] == 2.0)&(csv_data['who_adv'] == 1.0)].mean()
    MO = csv_data.loc[(csv_data['resource_level_1'] == 2.0)&(csv_data['who_adv'] == 2.0)].mean()
    MN = csv_data.loc[(csv_data['resource_level_1'] == 2.0)&(csv_data['who_adv'] == 3.0)].mean()
    
    LI = csv_data.loc[(csv_data['resource_level_1'] == 3.0)&(csv_data['who_adv'] == 1.0)].mean()
    LO = csv_data.loc[(csv_data['resource_level_1'] == 3.0)&(csv_data['who_adv'] == 2.0)].mean()
    LN = csv_data.loc[(csv_data['resource_level_1'] == 3.0)&(csv_data['who_adv'] == 3.0)].mean()
    
    #inequity_group 2x3 
    ModI = csv_data.loc[(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 1.0)].mean()
    ModO = csv_data.loc[(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 2.0)].mean()
    ModN = csv_data.loc[(csv_data['inequity_level'] == 1.0)&(csv_data['who_adv'] == 3.0)].mean()
    
    EI = csv_data.loc[(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 1.0)].mean()
    EO = csv_data.loc[(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 2.0)].mean()
    EN = csv_data.loc[(csv_data['inequity_level'] == 2.0)&(csv_data['who_adv'] == 3.0)].mean()
    
    #Main effect 3 levels 
    Small = csv_data.loc[(csv_data['resource_level_1'] == 1.0)].mean()
    Medium = csv_data.loc[(csv_data['resource_level_1'] == 2.0)].mean()
    Large = csv_data.loc[(csv_data['resource_level_1'] == 3.0)].mean()

    Moderate = csv_data.loc[(csv_data['inequity_level'] == 1.0)].mean()
    Extreme = csv_data.loc[(csv_data['inequity_level'] == 2.0)].mean()
    
    Ingroup = csv_data.loc[(csv_data['who_adv'] == 1.0)].mean()
    Outgroup = csv_data.loc[(csv_data['who_adv'] == 2.0)].mean()
    Neutral = csv_data.loc[(csv_data['who_adv'] == 3.0)].mean() 
    
    #combination
    ingroup_bias = Ingroup - Outgroup
    inequity_aversion = Moderate - Extreme
    interaction = ((SI -(SO+SN)/2) - ((MI+LI)/2 -(MO+LO+MN+LN)/4))
    overall_accept = (Extreme + Moderate)/2
    
    Table = pd.concat([SMI,SMO,SMN,SEI,SEO,SEN,MMI,MMO,MMN,MEI,MEO,MEN,LMI,LMO,LMN,LEI,LEO,LEN], axis =1)
    Table.columns = ['SMI','SMO','SMN','SEI','SEO','SEN','MMI','MMO','MMN','MEI','MEO','MEN','LMI','LMO','LMN','LEI','LEO','LEN']
    Table = Table.loc['reaction_time'].to_frame().T
    Table.index = [fileName[:-43]]
    sumDF_rt = pd.concat([sumDF_rt, Table])
    
    TB_18condition = pd.concat([SMI,SMO,SMN,SEI,SEO,SEN,MMI,MMO,MMN,MEI,MEO,MEN,LMI,LMO,LMN,LEI,LEO,LEN], axis =1).loc['zScore'].to_frame().T
    TB_18condition.columns = ['SMI','SMO','SMN','SEI','SEO','SEN','MMI','MMO','MMN','MEI','MEO','MEN','LMI','LMO','LMN','LEI','LEO','LEN']
    TB_18condition.index = [fileName[:-43]]
    DF_18condition = pd.concat([DF_18condition, TB_18condition])
    
    TB_resource_inequity = pd.concat([SM,SE,MM,ME,LM,LE], axis =1).loc['zScore'].to_frame().T
    TB_resource_inequity.columns = ['SM','SE','MM','ME','LM','LE']
    TB_resource_inequity.index = [fileName[:-43]]
    DF_resource_inequity = pd.concat([DF_resource_inequity, TB_resource_inequity])
    
    TB_resource_group = pd.concat([SI,SO,SN,MI,MO,MN,LI,LO,LN], axis =1).loc['zScore'].to_frame().T
    TB_resource_group.columns = ['SI','SO','SN','MI','MO','MN','LI','LO','LN']
    TB_resource_group.index = [fileName[:-43]]
    DF_resource_group = pd.concat([DF_resource_group, TB_resource_group])
    
    TB_inequity_group = pd.concat([ModI,ModO,ModN,EI,EO,EN], axis =1).loc['zScore'].to_frame().T
    TB_inequity_group.columns = ['ModI','ModO','ModN','EI','EO','EN']
    TB_inequity_group.index = [fileName[:-43]]
    DF_inequity_group = pd.concat([DF_inequity_group, TB_inequity_group])
    
    TB_maineffect = pd.concat([Small,Medium,Large,Moderate,Extreme,Ingroup,Outgroup,Neutral], axis =1).loc['zScore'].to_frame().T
    TB_maineffect.columns = ['Small','Medium','Large','Moderate','Extreme','Ingroup','Outgroup', 'Neutral']
    TB_maineffect.index = [fileName[:-43]]
    DF_maineffect = pd.concat([DF_maineffect, TB_maineffect])
    
    TB_combination = pd.concat([ingroup_bias,inequity_aversion,interaction,overall_accept], axis =1).loc['zScore'].to_frame().T
    TB_combination.columns = ['ingroup_bias', 'inequity_aversion', 'interaction', 'overall_accept']
    TB_combination.index = [fileName[:-43]]
    DF_combination = pd.concat([DF_combination, TB_combination])
    
    
with pd.ExcelWriter('RT_zScore_per_subj_condition[3MAD](n=58).xlsx') as writer:  
    DF_18condition.to_excel(writer, sheet_name='conditions(18)')
    DF_resource_inequity.to_excel(writer, sheet_name='resource_inequity')
    DF_resource_group.to_excel(writer, sheet_name='resource_group')
    DF_inequity_group.to_excel(writer, sheet_name='inequity_group')
    DF_maineffect.to_excel(writer, sheet_name='maineffect')
    DF_combination.to_excel(writer, sheet_name='combination')

<ipython-input-1-21b666d01df9>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_data['zScore'] = 0
C:\Users\jihhwan\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-1-21b666d01df9>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_data['zScore'].iloc[i] =  (csv

In [2]:
t

[4.649468699994031,
 0.9652331999968736,
 1.2626076000160538,
 1.1065567000187002,
 1.1824018000042995,
 1.256337300001178,
 1.0707244999939576,
 1.400655800011009,
 1.7062827000045218,
 1.2699285999988206,
 0.9144683000049554,
 1.3925111999851651,
 1.3298789000255056,
 1.9095897999941367,
 1.3064661000389606,
 1.4535648999735713,
 1.5711120999767445,
 1.276872399961576,
 1.2987029000069017,
 1.6886964999721386,
 1.2654794999980368,
 1.9285572000080717,
 1.999940000008792,
 3.985517299966887,
 1.9821040999959223,
 1.3630882000434212,
 1.1669094999670049,
 1.60057060001418,
 1.612051700009033,
 1.6404301000293344,
 1.2914529999834483,
 2.345646900008433,
 2.333203100017272,
 1.2129313000477848,
 0.9702601999742909,
 1.5558995000319555,
 0.8877235000254586,
 2.8795769999851477,
 2.053365900006611,
 1.5163546999683604,
 2.3403056999668483,
 1.4687588000088,
 1.3429804000188597,
 2.5744374000350945,
 1.4685565999825485,
 1.0706021000514738,
 1.3256672000279652,
 3.041067599959206,
 1.39244

In [5]:
Extreme

resource_level_1    2.000000
inequity_level      2.000000
who_adv             2.100000
response            0.400000
reaction_time       1.179931
zScore             -0.031629
dtype: float64